In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder, OneHotEncoder
import pandas as pd

In [22]:
data =pd.read_csv('sample_car_data.csv')
data


,Car_Name,Fuel_Type,Seller_Type,Transmission,Year,Kms_Driven,Selling_Price
0,CarA,Petrol,Dealer,Manual,2015,50000,4.5
1,CarB,Diesel,Individual,Automatic,2013,60000,3.2
2,CarC,CNG,Dealer,Manual,2018,25000,5.0
3,CarD,Diesel,Dealer,Automatic,2017,75000,2.8
4,CarE,Petrol,Individual,Manual,2020,30000,6.0


In [23]:
columns = ["Car_Name", "Fuel_Type", "Seller_Type","Transmission","Year", "Kms_Driven","Selling_Price"]
for col in columns:
    print(f"Unique values in '{col}':")
    print(data[col].unique())
    print("-" * 40)


Unique values in 'Car_Name':
['CarA' 'CarB' 'CarC' 'CarD' 'CarE']
----------------------------------------
Unique values in 'Fuel_Type':
['Petrol' 'Diesel' 'CNG']
----------------------------------------
Unique values in 'Seller_Type':
['Dealer' 'Individual']
----------------------------------------
Unique values in 'Transmission':
['Manual' 'Automatic']
----------------------------------------
Unique values in 'Year':
[2015 2013 2018 2017 2020]
----------------------------------------
Unique values in 'Kms_Driven':
[50000 60000 25000 75000 30000]
----------------------------------------
Unique values in 'Selling_Price':
[4.5 3.2 5.  2.8 6. ]
----------------------------------------


In [24]:
data

,Car_Name,Fuel_Type,Seller_Type,Transmission,Year,Kms_Driven,Selling_Price
0,CarA,Petrol,Dealer,Manual,2015,50000,4.5
1,CarB,Diesel,Individual,Automatic,2013,60000,3.2
2,CarC,CNG,Dealer,Manual,2018,25000,5.0
3,CarD,Diesel,Dealer,Automatic,2017,75000,2.8
4,CarE,Petrol,Individual,Manual,2020,30000,6.0


In [25]:
updates=['Car_Name','Fuel_Type','Seller_Type','Transmission','Year','Kms_Driven','Selling_Price']
one =OneHotEncoder()
hot_code = one.fit_transform(data[updates]).toarray()
one.get_feature_names_out(data[updates].columns)
new_df = pd.DataFrame(hot_code, columns=one.get_feature_names_out(data[updates].columns))
data =pd.concat([data.drop(updates, axis=1),new_df],axis=1)

In [26]:
data

,Car_Name_CarA,Car_Name_CarB,Car_Name_CarC,Car_Name_CarD,Car_Name_CarE,Fuel_Type_CNG,Fuel_Type_Diesel,Fuel_Type_Petrol,Seller_Type_Dealer,Seller_Type_Individual,...,Kms_Driven_25000,Kms_Driven_30000,Kms_Driven_50000,Kms_Driven_60000,Kms_Driven_75000,Selling_Price_2.8,Selling_Price_3.2,Selling_Price_4.5,Selling_Price_5.0,Selling_Price_6.0
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [27]:
import pickle
with open('onehotencode', 'wb') as file:
    pickle.dump(one, file)

In [28]:
new_selling_price = ['Selling_Price_2.8','Selling_Price_3.2','Selling_Price_4.5','Selling_Price_5.0','Selling_Price_6.0']
X=data.drop(new_selling_price, axis=1)

In [29]:
y= data[new_selling_price]

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scl =StandardScaler()
train_X =scl.fit_transform(X_train)
test_X =scl.transform(X_test)

In [31]:
(X.shape[1],)

(22,)

In [54]:
import tensorflow 
from tensorflow.keras.models import Sequential
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.layers import Dense
def new_model(neurons=64,layers=1):
    model= Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(X.shape[1],)))
    for _ in range(layers-1):
        model.add(Dense(neurons, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

In [55]:
model =KerasRegressor(layers=1,neurons=32,build_fn=new_model,verbose=1)
param_grid={
    'neurons':[16,32,64,128],
    'layers':[1,2],
    'epochs':[50,100]}



In [59]:
gs =GridSearchCV(estimator=model,param_grid=param_grid,cv=3,verbose=2,n_jobs=-1)
gs_result=gs.fit(train_X,y_train)
print("Best: %f using %s" % (gs_result.best_score_, gs_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits
Epoch 1/50


c:\Users\dussa\OneDrive\Desktop\Python\Krish\.conda\lib\site-packages\sklearn\model_selection\_search.py:1135: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan]
  warnings.warn(
c:\Users\dussa\OneDrive\Desktop\Python\Krish\.conda\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\dussa\OneDrive\Desktop\Python\Krish\.conda\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 933ms/step - loss: 0.8277 - mae: 0.7579
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 0.7690 - mae: 0.7287
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step - loss: 0.7136 - mae: 0.6997
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - loss: 0.6638 - mae: 0.6722
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - loss: 0.6175 - mae: 0.6507
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.5740 - mae: 0.6287
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.5332 - mae: 0.6065
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.4955 - mae: 0.5845
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.4598 - mae: 0.5622
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.4271 - mae: 0.5402
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.3969 - mae: 0.5184
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.3692 - mae: 0.4994
Epoch 13/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step - loss: 0.3439 - mae

In [62]:
reg_model = Sequential([Dense(16, activation='relu', input_shape=(X.shape[1],)),
                        Dense(16, activation='relu'),
                        Dense(1)])

c:\Users\dussa\OneDrive\Desktop\Python\Krish\.conda\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [66]:
reg_model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 16)             │           368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 657 (2.57 KB)

 Trainable params: 657 (2.57 KB)

 Non-trainable params: 0 (0.00 B)

In [65]:
reg_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [75]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime
log_dir ='reg/'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensor =TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reg_model.fit(train_X, y_train, validation_data=(test_X, y_test), epochs=100, batch_size=32, callbacks=[tensor, early_stopping])

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step - loss: 0.2550 - mae: 0.4476 - val_loss: 0.1603 - val_mae: 0.3100
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step - loss: 0.2404 - mae: 0.4315 - val_loss: 0.1607 - val_mae: 0.3044
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - loss: 0.2262 - mae: 0.4137 - val_loss: 0.1614 - val_mae: 0.2979
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - loss: 0.2131 - mae: 0.3949 - val_loss: 0.1624 - val_mae: 0.2909
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step - loss: 0.2011 - mae: 0.3754 - val_loss: 0.1637 - val_mae: 0.2834
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step - loss: 0.1906 - mae: 0.3555 - val_loss: 0.1653 - val_mae: 0.2763
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step - loss: 0.1818 - mae: 0.3427 - val_loss: 0.1671 - val_mae: 0.2693
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step - loss: 0.1748 - mae: 0.3355 - val_loss: 0.1692 - val_mae: 0.2626
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step - loss: 0.169

In [76]:
reg_model.save('regexp_model.h5')

In [81]:
%load_ext tensorboard
%tensorboard --logdir reg

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 5884), started 0:00:17 ago. (Use '!kill 5884' to kill it.)